In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/집중교육/팀프로젝트/

/content/drive/MyDrive/집중교육/팀프로젝트


In [3]:
from typing import Sequence
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
import os
import random
import numpy as np
import pickle

random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
englabel = ['The Eiffel Tower', 'The Great Wall of China', 'The Mona Lisa', 'aircraft carrier', 'airplane', 'alarm clock', 'ambulance', 'angel', 'animal migration', 'ant', 'anvil', 'apple', 'arm', 'asparagus', 'axe', 'backpack', 'banana', 'bandage', 'barn', 'baseball bat', 'baseball', 'basket', 'basketball', 'bat', 'bathtub', 'beach', 'bear', 'beard', 'bed', 'bee', 'belt', 'bench', 'bicycle', 'binoculars', 'bird', 'birthday cake', 'blackberry', 'blueberry', 'book', 'boomerang', 'bottlecap', 'bowtie', 'bracelet', 'brain', 'bread', 'bridge', 'broccoli', 'broom', 'bucket', 'bulldozer', 'bus', 'bush', 'butterfly', 'cactus', 'cake', 'calculator', 'calendar', 'camel', 'camera', 'camouflage', 'campfire', 'candle', 'cannon', 'canoe', 'car', 'carrot', 'castle', 'cat', 'ceiling fan', 'cell phone', 'cello', 'chair', 'chandelier', 'church', 'circle', 'clarinet', 'clock', 'cloud', 'coffee cup', 'compass', 'computer', 'cookie', 'cooler', 'couch', 'cow', 'crab', 'crayon', 'crocodile', 'crown', 'cruise ship', 'cup', 'diamond', 'dishwasher', 'diving board', 'dog', 'dolphin', 'donut', 'door', 'dragon', 'dresser', 'drill', 'drums', 'duck', 'dumbbell', 'ear', 'elbow', 'elephant', 'envelope', 'eraser', 'eye', 'eyeglasses', 'face', 'fan', 'feather', 'fence', 'finger', 'fire hydrant', 'fireplace', 'firetruck', 'fish', 'flamingo', 'flashlight', 'flip flops', 'floor lamp', 'flower', 'flying saucer', 'foot', 'fork', 'frog', 'frying pan', 'garden hose', 'garden', 'giraffe', 'goatee', 'golf club', 'grapes', 'grass', 'guitar', 'hamburger', 'hammer', 'hand', 'harp', 'hat', 'headphones', 'hedgehog', 'helicopter', 'helmet', 'hexagon', 'hockey puck', 'hockey stick', 'horse', 'hospital', 'hot air balloon', 'hot dog', 'hot tub', 'hourglass', 'house plant', 'house', 'hurricane', 'ice cream', 'jacket', 'jail', 'kangaroo', 'key', 'keyboard', 'knee', 'knife', 'ladder', 'lantern', 'laptop', 'leaf', 'leg', 'light bulb', 'lighter', 'lighthouse', 'lightning', 'line', 'lion', 'lipstick', 'lobster', 'lollipop', 'mailbox', 'map', 'marker', 'matches', 'megaphone', 'mermaid', 'microphone', 'microwave', 'monkey', 'moon', 'mosquito', 'motorbike', 'mountain', 'mouse', 'moustache', 'mouth', 'mug', 'mushroom', 'nail', 'necklace', 'nose', 'ocean', 'octagon', 'octopus', 'onion', 'oven', 'owl', 'paint can', 'paintbrush', 'palm tree', 'panda', 'pants', 'paper clip', 'parachute', 'parrot', 'passport', 'peanut', 'pear', 'peas', 'pencil', 'penguin', 'piano', 'pickup truck', 'picture frame', 'pig', 'pillow', 'pineapple', 'pizza', 'pliers', 'police car', 'pond', 'pool', 'popsicle', 'postcard', 'potato', 'power outlet', 'purse', 'rabbit', 'raccoon', 'radio', 'rain', 'rainbow', 'rake', 'remote control', 'rhinoceros', 'rifle', 'river', 'roller coaster', 'rollerskates', 'sailboat', 'sandwich', 'saw', 'saxophone', 'school bus', 'scissors', 'scorpion', 'screwdriver', 'sea turtle', 'see saw', 'shark', 'sheep', 'shoe', 'shorts', 'shovel', 'sink', 'skateboard', 'skull', 'skyscraper', 'sleeping bag', 'smiley face', 'snail', 'snake', 'snorkel', 'snowflake', 'snowman', 'soccer ball', 'sock', 'speedboat', 'spider', 'spoon', 'spreadsheet', 'square', 'squiggle', 'squirrel', 'stairs', 'star', 'steak', 'stereo', 'stethoscope', 'stitches', 'stop sign', 'stove', 'strawberry', 'streetlight', 'string bean', 'submarine', 'suitcase', 'sun', 'swan', 'sweater', 'swing set', 'sword', 'syringe', 't-shirt', 'table', 'teapot', 'teddy-bear', 'telephone', 'television', 'tennis racquet', 'tent', 'tiger', 'toaster', 'toe', 'toilet', 'tooth', 'toothbrush', 'toothpaste', 'tornado', 'tractor', 'traffic light', 'train', 'tree', 'triangle', 'trombone', 'truck', 'trumpet', 'umbrella', 'underwear', 'van', 'vase', 'violin', 'washing machine', 'watermelon', 'waterslide', 'whale', 'wheel', 'windmill', 'wine bottle', 'wine glass', 'wristwatch', 'yoga', 'zebra', 'zigzag']

In [5]:
penPositions = []
labels = []

if not os.path.exists('./sketchrnn/penPositions.pkl'):
    directory = './top10000'

    data_size = 9000
    for filename in os.listdir(directory):
        if filename.endswith(".ndjson"):
            filePath = os.path.join(directory, filename)

            print(f"Processing file: {filePath}")
            df = pd.read_json(filePath, lines=True)


            penPosition = df['drawing'].tolist()

            if len(penPosition) >= data_size:
                penPos = random.sample(penPosition, data_size)
            else:
                penPos = penPosition

            penPositions.extend(penPos)
            numLabel = englabel.index(filename.split('.')[0])
            labels.extend([numLabel] * len(penPos))

    # pickle을 사용해 penPositions와 labels를 저장
    with open('./sketchrnn/penPositions.pkl', 'wb') as f:
        pickle.dump(penPositions, f)

    with open('./sketchrnn/labels.pkl', 'wb') as f:
        pickle.dump(labels, f)

else:
    with open('./sketchrnn/penPositions.pkl', 'rb') as f:
        penPositions = pickle.load(f)

    with open('./sketchrnn/labels.pkl', 'rb') as f:
        labels = pickle.load(f)

In [10]:
print(len(penPositions))
print(penPositions[0])

3105000
[[[66, 63, 49, 50], [210, 162, 92, 75]], [[47, 22, 10, 1, 0, 7, 16, 21, 25, 33, 40, 48, 60, 64, 63, 54, 57, 68, 82, 90, 96, 94, 84, 75, 85, 98, 106, 108, 104, 89, 58, 49, 46], [79, 74, 66, 49, 36, 18, 13, 22, 45, 14, 6, 0, 4, 13, 30, 43, 33, 21, 15, 14, 24, 36, 54, 61, 52, 51, 56, 65, 77, 89, 99, 98, 93]], [[137, 124, 143, 165, 179, 193, 196, 200, 205, 222, 226, 232, 244, 247], [202, 79, 126, 158, 99, 66, 67, 79, 122, 48, 53, 78, 153, 190]], [[250, 255], [220, 232]]]


In [18]:
max_x = 0
max_y = 0

# 각 좌표 리스트의 모든 값들을 4로 나누고 정수형으로 변환
for data in penPositions:
    for stroke in data:
        # 리스트의 각 요소를 4로 나누고 정수로 변환
        stroke[0] = [value // 4 for value in stroke[0]]
        stroke[1] = [value // 4 for value in stroke[1]]

        # 나눠진 리스트에서 최댓값을 구함
        if max(stroke[0]) > max_x:
            max_x = max(stroke[0])
        if max(stroke[1]) > max_y:
            max_y = max(stroke[1])

print(max_x, max_y)


63 63


In [16]:
print(penPositions[0])

[[[66, 63, 49, 50], [210, 162, 92, 75]], [[47, 22, 10, 1, 0, 7, 16, 21, 25, 33, 40, 48, 60, 64, 63, 54, 57, 68, 82, 90, 96, 94, 84, 75, 85, 98, 106, 108, 104, 89, 58, 49, 46], [79, 74, 66, 49, 36, 18, 13, 22, 45, 14, 6, 0, 4, 13, 30, 43, 33, 21, 15, 14, 24, 36, 54, 61, 52, 51, 56, 65, 77, 89, 99, 98, 93]], [[137, 124, 143, 165, 179, 193, 196, 200, 205, 222, 226, 232, 244, 247], [202, 79, 126, 158, 99, 66, 67, 79, 122, 48, 53, 78, 153, 190]], [[250, 255], [220, 232]]]


In [ ]:
def penPosToSequence(penPos):
    sequence = []
    for stroke in range(len(penPos)):
      x = penPos[stroke][0]
      y = penPos[stroke][1]
      for i in range(len(x)):
        if i == 0:
          penState = 0
        elif i == len(x) - 1:
          penState = 2
        else:
          penState = 1
        sequence.append([x[i], y[i], penState])
    return sequence


class PenDataset(Dataset):
    def __init__(self, pen_positions, labels):
        """
        Args:
            pen_positions (list of lists): 각 시퀀스에 해당하는 펜 위치 데이터 리스트
            labels (list): 시퀀스에 해당하는 레이블 리스트
        """
        self.pen_positions = pen_positions
        self.labels = labels

    def __len__(self):
        # 데이터셋의 총 길이 반환
        return len(self.pen_positions)

    def __getitem__(self, idx):
        pen_position = self.pen_positions[idx]
        sequence = penPosToSequence(pen_position)
        sequence_tensor = torch.tensor(sequence, dtype=torch.float32)

        label = self.labels[idx]
        label_tensor = torch.tensor(label, dtype=torch.long)

        return sequence_tensor, label_tensor

def collate_fn(batch):
    # 시퀀스와 레이블을 분리
    sequences, labels = zip(*batch)

    # 시퀀스에 대해 패딩 적용
    padded_sequences = pad_sequence(sequences, batch_first=True)  # 배치 내 시퀀스를 동일한 길이로 패딩

    # 레이블은 패딩이 필요 없으므로 텐서로 변환
    labels = torch.tensor(labels, dtype=torch.long)

    return padded_sequences, labels

dataset = PenDataset(penPositions, labels)

dataset_size = len(dataset)
train_size = int(dataset_size * 0.7)
validation_size = int(dataset_size * 0.2)
test_size = dataset_size - train_size - validation_size


print(f"Dataset size: {dataset_size}")
print(f"Training set size: {train_size}")
print(f"Validation set size: {validation_size}")
print(f"Test set size: {test_size}")

# random_split을 사용해 데이터셋을 훈련 및 검증 데이터셋으로 분할
train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])

# 나눠진 데이터셋을 DataLoader에 전달
trainloader = DataLoader(train_dataset, batch_size=100, shuffle=True, collate_fn=collate_fn)
validationloader = DataLoader(validation_dataset, batch_size=100, shuffle=False, collate_fn=collate_fn)
testloader = DataLoader(test_dataset, batch_size=100, shuffle=False, collate_fn=collate_fn)

Dataset size: 3105000
Training set size: 2173500
Validation set size: 621000
Test set size: 310500


In [ ]:
# import matplotlib.pyplot as plt

# def plot_pen_position(sequence, title):
#     """
#     penPosition 시퀀스를 기반으로 그래프를 그리는 함수.
#     각 시퀀스는 [x, y, penState]로 구성된다고 가정.
#     penState가 0: 펜이 처음, 1: 펜이 움직이는 중, 2: 펜이 들린 상태.
#     """
#     x = []
#     y = []

#     for i in range(len(sequence)):
#         current_x, current_y, penState = sequence[i]

#         if penState == 0:
#             x = [current_x]
#             y = [current_y]
#         elif penState == 1:
#             x.append(current_x)
#             y.append(current_y)
#         elif penState == 2:
#             x.append(current_x)
#             y.append(current_y)
#             plt.plot(x, y, 'b-')
#             x, y = [], []

#     plt.gca().invert_yaxis()
#     plt.title(title)
#     plt.show()


# imgCount = 30
# random_img_idx = random.sample(range(len(penPositions)), imgCount)

# for i in random_img_idx:
#     pen_position_sample = penPosToSequence(penPositions[i])
#     plot_pen_position(pen_position_sample, englabel[labels[i]])



In [ ]:
print(labels[2998: 3002])

[131, 131, 131, 131]


In [ ]:
class HParams():
    def __init__(self):
        self.enc_hidden_size = 2048
        self.Nz = 1024
        self.num_classes = 345
        self.overfitting_count = 10
        self.lr = 0.001
        self.epochs = 30

        # self.M = 20
        self.dropout = 0.9
        # self.batch_size = 100
        # self.eta_min = 0.01
        # self.R = 0.99995
        # self.KL_min = 0.2
        # self.wKL = 0.5

        # self.lr_decay = 0.9999
        # self.min_lr = 0.00001
        # self.grad_clip = 1.
        # self.temperature = 0.4
        # self.max_seq_length = 200


hp = HParams()

class EncoderRNN(nn.Module):
    def __init__(self):
        super(EncoderRNN, self).__init__()
        self.lstm = nn.LSTM(3, hp.enc_hidden_size, dropout = hp.dropout, bidirectional=True, batch_first=True)  # 입력 크기 3으로 설정
        self.fc_mu = nn.Linear(2 * hp.enc_hidden_size, hp.Nz)
        self.fc_sigma = nn.Linear(2 * hp.enc_hidden_size, hp.Nz)

    def forward(self, inputs, batch_size, hidden_cell=None):
        if hidden_cell is None:
            hidden = torch.zeros(2, batch_size, hp.enc_hidden_size).cuda()
            cell = torch.zeros(2, batch_size, hp.enc_hidden_size).cuda()
            hidden_cell = (hidden, cell)

        # LSTM에 시퀀스를 입력
        _, (hidden, _) = self.lstm(inputs.float(), hidden_cell)

        # 양방향 LSTM의 출력을 결합
        hidden_forward, hidden_backward = torch.split(hidden, 1, 0)
        hidden_cat = torch.cat([hidden_forward.squeeze(0), hidden_backward.squeeze(0)], 1)

        # mu와 sigma를 계산하여 잠재 벡터 z 생성
        mu = self.fc_mu(hidden_cat)
        sigma_hat = self.fc_sigma(hidden_cat)
        sigma = torch.exp(sigma_hat / 2.)
        z = mu + sigma * torch.randn_like(mu)

        return z, mu, sigma_hat


class Classifier(nn.Module):
    def __init__(self, num_classes):
        super(Classifier, self).__init__()
        self.encoder = EncoderRNN()  # 인코더 (시퀀스 데이터를 인코딩)
        self.fc = nn.Linear(hp.Nz, num_classes)  # 잠재 벡터 z를 분류하기 위한 fully connected layer

    def forward(self, inputs):
        batch_size = inputs.size(0)
        z, mu, sigma_hat = self.encoder(inputs, batch_size)  # 인코더 호출, z는 잠재 벡터
        logits = self.fc(z)  # 잠재 벡터 z를 분류기 입력으로 사용
        return logits





In [ ]:
from tqdm import tqdm

# 모델 생성
model = Classifier(num_classes=hp.num_classes).to(device)

# 손실 함수와 옵티마이저 설정
criterion = CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=hp.lr)

best_loss = float('inf')
overfitting_count = 0
# 학습 루프
for epoch in range(hp.epochs):
    model.train()
    running_loss = 0.0

    for i, (inputs, labels) in enumerate(tqdm(trainloader, desc=f"Epoch {epoch+1}/{hp.epochs}", unit="batch")):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)

        loss = criterion(outputs, labels)

        loss.backward()

        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {running_loss / len(trainloader)}')

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(validationloader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    if val_loss < best_loss:
        best_loss = val_loss
        overfitting_count = 0

        fileName = f'./sketchrnn/model.pth'
        torch.save(model.state_dict(), fileName)
    else:
        overfitting_count += 1
        if overfitting_count >= hp.overfitting_count:
            print(f'Early stopping at epoch {epoch+1}')
            break

print("학습 완료!")

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:88: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.9 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
Epoch 1/30: 100%|██████████| 21735/21735 [22:29<00:00, 16.10batch/s]


Epoch 1, Loss: 1.8297791108444474


Epoch 2/30: 100%|██████████| 21735/21735 [22:27<00:00, 16.13batch/s]


Epoch 2, Loss: 1.3213129296175552


Epoch 3/30: 100%|██████████| 21735/21735 [22:24<00:00, 16.16batch/s]


...

Epoch 15, Loss: 3.0103419450773488


Epoch 16/30: 100%|██████████| 21735/21735 [22:32<00:00, 16.07batch/s]


Epoch 16, Loss: 2.4046542425772115


Epoch 17/30: 100%|██████████| 21735/21735 [22:32<00:00, 16.07batch/s]


Epoch 17, Loss: 1.8304976949917742
Early stopping at epoch 17
학습 완료!


In [ ]:
finalmodel = Classifier(num_classes=hp.num_classes).to(device)
finalmodel.load_state_dict(torch.load('./sketchrnn/model.pth'))
finalmodel.eval()
# 테스트 정확도
correct = 0
total = 0
with torch.no_grad():
    for i, (inputs, labels) in enumerate(testloader):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = finalmodel(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the on the test images: {100 * correct / total:.2f}%')


<ipython-input-11-290d83947099>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  finalmodel.load_state_dict(torch.load('./sketchrnn/model.pth'))


Accuracy of the on the test images: 70.98%
